In [ ]:
import datetime
from itertools import zip_longest
import os
import re
import sys

import pandas as pd

import muscope_loader

In [ ]:
print(sys.modules['muscope_loader'].__file__)
muscope_loader_dp = os.path.dirname(sys.modules['muscope_loader'].__file__)
downloads_dp = os.path.join(muscope_loader_dp, 'downloads')
dyhrman_ms_xls_fp = os.path.join(downloads_dp, 'Dyhrman_MS_incubation_assoc_data_v3.xls')

print(dyhrman_ms_xls_fp)
os.path.exists(dyhrman_ms_xls_fp)


In [ ]:
def grouper(iterable, n, fillvalue=None):
    "Collect data into fixed-length chunks or blocks"
    # grouper('ABCDEFG', 3, 'x') --> ABC DEF Gxx"
    args = [iter(iterable)] * n
    return zip_longest(*args, fillvalue=fillvalue)


In [ ]:
# skip rows 0, 2
# row 1 becomes row 0
# now the header is row 0
core_attr_plus_data_df = pd.read_excel(
    dyhrman_ms_xls_fp,
    sheet_name='core attributes + data',
    skiprows=(0,2)
)
##print(core_attr_plus_data_df.head())

# there are 4 seq_names for each sample name
# the first 2 seq_names are paired-end reads e.g. 
#   SM178_S9_L001_R1_001.fastq.gz and SM178_S9_L001_R2_001.fastq.gz
# the second 2 seq_names are paired-end reads e.g.
#   SM226_S9_L005_R1_001.fastq.gz and SM226_S9_L005_R2_001.fastq.gz

# this re will extract a new sample name from each R1 seq_name
new_sample_name_re = re.compile(r'^(?P<new_sample_name>SM\d+_S\d+)_L\d+_R[12]_\d+\.fastq\.gz$')

# parse time strings such as '12:45:00' and '1245'
# the first 3 collection times are datetime objects, the remaining collection times are just strings like "1205"
time_re = re.compile(r'^(?P<hour>\d{1,2}):?(?P<minute>\d{1,2})(:(?P<second>)\d{1,2})?$')

# the old sample_names will go into sample_description_column
# a sample_description will be entered for the 3rd row in addition to the 1st row
##sample_description_column = []
for (r1, row1), (r2, row2), (r3, row3), (r4, row4) in grouper(core_attr_plus_data_df.iterrows(), n=4):
    # change the cruise name to MESO-SCOPE
    core_attr_plus_data_df.loc[row1.name, 'cruise_name'] = 'MESO-SCOPE'
    core_attr_plus_data_df.loc[row2.name, 'cruise_name'] = 'MESO-SCOPE'
    core_attr_plus_data_df.loc[row3.name, 'cruise_name'] = 'MESO-SCOPE'
    core_attr_plus_data_df.loc[row4.name, 'cruise_name'] = 'MESO-SCOPE'
    
    ##print(row1.seq_name)
    ##sample_description_column.append(row1.sample_name)  # e.g. insitu_15m_S9C1_rep1
    ##sample_description_column.append(row2.sample_name)  # empty
    ##sample_description_column.append(row1.sample_name)  # e.g insitu_15m_S9C1_rep1
    ##sample_description_column.append(row2.sample_name)  # empty

    # extract a new sample name from each R1 seq_name
    new_sample_name_1 = new_sample_name_re.search(row1.seq_name).group('new_sample_name')
    new_sample_name_3 = new_sample_name_re.search(row3.seq_name).group('new_sample_name')

    # convert the strings in collection_time to datetime.time objects
    ##print('"{}"'.format(row1.collection_time))
    collection_time_match = time_re.search(str(row1.collection_time))
    core_attr_plus_data_df.loc[row1.name, 'collection_time'] = datetime.time(
        hour=int(collection_time_match.group('hour')),
        minute=int(collection_time_match.group('minute')))
    
    core_attr_plus_data_df.loc[row1.name, 'sample_name'] = new_sample_name_1
    core_attr_plus_data_df.loc[row3.name, 'sample_name'] = new_sample_name_3

    # copy attributes from 1st row to 3rd row
    core_attr_plus_data_df.loc[row3.name, 0:8] = core_attr_plus_data_df.iloc[row1.name, 0:8]
    core_attr_plus_data_df.loc[row3.name, 10:] = core_attr_plus_data_df.iloc[row1.name, 10:]
    
core_attr_plus_data_df['sample_description'] = sample_description_column
core_attr_plus_data_df